In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import json
import requests
from ps_wd import pswd
from api_key import apikey
from pandas import json_normalize
import datetime 

### ETL of Dow Jones data

In [2]:
#DowJones data for years 2016-2020 is downloaded as a csv file from Yahoo Finance
#Read the csv file 
dow_csv = "Resources/DowJonesIndex.csv"
dow_df = pd.read_csv(dow_csv)
dow_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1/8/2016,16519.16992,16651.89063,16314.57031,16346.45020,16346.45020,141850000
1,1/11/2016,16358.70996,16461.84961,16232.03027,16398.57031,16398.57031,127790000
2,1/12/2016,16419.10938,16591.34961,16322.07031,16516.22070,16516.22070,117480000
3,1/13/2016,16526.63086,16593.50977,16123.20020,16151.41016,16151.41016,153530000
4,1/14/2016,16159.00977,16482.05078,16075.12012,16379.04981,16379.04981,158830000


In [3]:
dow_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [4]:
#Round to 3 decimal places
dow_df = dow_df.round(3)

In [5]:
#Only Date, Open & Close are needed for our analysis
#Create a dataframe with the columns required
new_dow_df = dow_df[['Date', 'Open', 'Close']].copy()
new_dow_df.head()

,Date,Open,Close
0,1/8/2016,16519.170,16346.450
1,1/11/2016,16358.710,16398.570
2,1/12/2016,16419.109,16516.221
3,1/13/2016,16526.631,16151.410
4,1/14/2016,16159.010,16379.050


In [6]:
#Calculate the daily change in the index
new_dow_df['Change'] = new_dow_df['Open'] - new_dow_df['Close']
new_dow_df.head()

,Date,Open,Close,Change
0,1/8/2016,16519.170,16346.450,172.720
1,1/11/2016,16358.710,16398.570,-39.860
2,1/12/2016,16419.109,16516.221,-97.112
3,1/13/2016,16526.631,16151.410,375.221
4,1/14/2016,16159.010,16379.050,-220.040


In [7]:
#Rename a column 
new_dow_df = new_dow_df.rename(columns={'Date': 'Dates'})
new_dow_df.head()

,Dates,Open,Close,Change
0,1/8/2016,16519.170,16346.450,172.720
1,1/11/2016,16358.710,16398.570,-39.860
2,1/12/2016,16419.109,16516.221,-97.112
3,1/13/2016,16526.631,16151.410,375.221
4,1/14/2016,16159.010,16379.050,-220.040


In [8]:
#Change the 'Dates' column datatype from string to date using to_datetime
new_dow_df['Dates']= pd.to_datetime(new_dow_df['Dates'])
new_dow_df.head()

,Dates,Open,Close,Change
0,2016-01-08,16519.170,16346.450,172.720
1,2016-01-11,16358.710,16398.570,-39.860
2,2016-01-12,16419.109,16516.221,-97.112
3,2016-01-13,16526.631,16151.410,375.221
4,2016-01-14,16159.010,16379.050,-220.040


In [9]:
new_dow_df.dtypes

Dates     datetime64[ns]
Open             float64
Close            float64
Change           float64
dtype: object

In [10]:
#Call the dt.date attribute to get just the date portion from datetime format
new_dow_df['Dates'] = pd.to_datetime(new_dow_df['Dates']).dt.date
new_dow_df.head()

,Dates,Open,Close,Change
0,2016-01-08,16519.170,16346.450,172.720
1,2016-01-11,16358.710,16398.570,-39.860
2,2016-01-12,16419.109,16516.221,-97.112
3,2016-01-13,16526.631,16151.410,375.221
4,2016-01-14,16159.010,16379.050,-220.040


In [11]:
#!pip install psycopg2

In [12]:
#Establish connection with the database
engine = create_engine('postgresql://postgres:'+ pswd + '@localhost:5432/cnn_dow_crime_db')
connection = engine.connect()


In [13]:
#Load the data to SQL database (a table named 'dow_jones' is created in postgresql)
new_dow_df.to_sql(name='dow_jones', con=engine, if_exists='replace', index=False)

In [14]:
#Query the SQL database
pd.read_sql_query('select * from dow_jones', con=engine).head()

,Dates,Open,Close,Change
0,2016-01-08,16519.170,16346.450,172.720
1,2016-01-11,16358.710,16398.570,-39.860
2,2016-01-12,16419.109,16516.221,-97.112
3,2016-01-13,16526.631,16151.410,375.221
4,2016-01-14,16159.010,16379.050,-220.040


## ETL of CNN headlines

### ETL of one webpage

In [15]:
# URL of page to be scraped
url = 'https://www.cnn.com/article/sitemap-2016-1.html'

In [16]:
# Retrieve page with the requests module
response = requests.get(url)

In [17]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [18]:
# Extract the required headlines 
title_results = soup.find_all('span', class_="sitemap-link")
#title_results

# Loop through returned results to extract only the text 
title_list = []
for result in title_results:
    title = result.text
    title_list.append(title)

In [19]:
len(title_list)

2238

In [20]:
#Check to see if the last title is correct (cross-check on webpage)
title_list[2237]

"China says it's building new homegrown aircraft carrier"

In [21]:
#Create a dataframe for the titles
title_df = DataFrame(title_list, columns =['Title'])
title_df.head()

,Title
0,Title
1,McCain on Cruz claim: 'It's an outright lie'
2,Terry Wogan: Veteran broadcaster dies
3,9 great quotes about the Iowa caucuses
4,Missing teen found dead; Virginia Tech student...


In [23]:
#Drop the first row
title_df.drop(title_df.index[0]).head()

,Title
1,McCain on Cruz claim: 'It's an outright lie'
2,Terry Wogan: Veteran broadcaster dies
3,9 great quotes about the Iowa caucuses
4,Missing teen found dead; Virginia Tech student...
5,Jerry Falwell Jr. cites his father in Trump en...


In [24]:
# Extract dates for the titles
date_results = soup.find_all('span', class_="date")

# Loop through returned results to extract text
date_list = []
for result in date_results:
    date = result.text
    date_list.append(date)    

In [25]:
#Create dataframe for dates
date_df = DataFrame(date_list, columns =['Date'])
date_df.head()

,Date
0,Date
1,2016-01-31
2,2016-01-31
3,2016-01-31
4,2016-01-31


In [26]:
#Drop the first row
date_df.drop(date_df.index[0]).head()

,Date
1,2016-01-31
2,2016-01-31
3,2016-01-31
4,2016-01-31
5,2016-01-31


### Extract multiple webpages

In [27]:
# Build the URL for all the years and months of data to be extracted
#url = 'https://www.cnn.com/article/sitemap-' + year + '-' + month + '.html'

In [28]:
#Extracting data from multiple pages for 2016-2020
years = np.arange(2016, 2021)
months = np.arange(1, 13)

title_list = []
date_list = []

for year in years:
    for month in months: 
  
        response = requests.get("https://www.cnn.com/article/sitemap-" + str(year) + '-' + str(month) + ".html")
  
        soup = BeautifulSoup(response.text, 'html.parser')
  
        date_results= soup.find_all('span', class_="date")
        title_results = soup.find_all('span', class_="sitemap-link")
        
        i = 0
        for result in title_results:
            title = result.text
            #The find() method returns the index of first occurrence of the substring (if found). If not found, it returns -1
            if title.find ('Trump') != -1: 
                title_list.append(title)
                #Append the date corresponding to the "Trump" title extracted
                date_list.append(date_results[i].text)
            i+=1

In [29]:
#title_list

In [30]:
len(title_list)

26723

In [40]:
title_list[26722]

"GOP silence on Trump's false election claims recalls McCarthy era"

In [41]:
#date_list

In [42]:
#Create a dataframe for the dates and titles
cnn_df = pd.DataFrame({"Dates": date_list, "Headlines":title_list})
cnn_df.head()

,Dates,Headlines
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


In [43]:
# convert the 'Dates' column datatype to datetime format
cnn_df['Dates']= pd.to_datetime(cnn_df['Dates'])
cnn_df.head()

,Dates,Headlines
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


In [44]:
#Call the dt.date attribute to get just the date portion from datetime format
cnn_df['Dates'] = pd.to_datetime(cnn_df['Dates']).dt.date
cnn_df.head()

,Dates,Headlines
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


In [45]:
#Load the data into SQL database
cnn_df.to_sql(name='news_titles', con=engine, if_exists='replace', index=False)

In [46]:
#Query the SQL database
pd.read_sql_query('select * from news_titles', con=engine).head()

,Dates,Headlines
0,2016-01-31,Jerry Falwell Jr. cites his father in Trump en...
1,2016-01-31,"CNN/WMUR poll: Trump, Sanders still up in New ..."
2,2016-01-31,"Major Iowa poll shows Trump, Clinton holding n..."
3,2016-01-31,Trump on blue collar appeal: My private jet is...
4,2016-01-31,Why these Iowans are choosing Donald Trump ove...


### ETL of crime data

In [47]:
#Got the crime data in json format using API for FBI data
# Save config information
crime_url = "https://api.usa.gov/crime/fbi/sapi/api/data/supplemental/states/CA/property_type/2016/2019?API_KEY=" + apikey

crime_response = requests.get(crime_url)
crime_json = crime_response.json()
#print(json.dumps(crime_json, indent=4, sort_keys=True))


In [48]:
# import json_normalize
# in this dataset, the data to extract is under 'results'
crime_df = json_normalize(crime_json, 'results')
crime_df

,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,currency_notes_etc,...,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count,data_year
0,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349,2018
1,8904,8904,0,8904,8904,8904,8904,8904,8904,8904,...,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290,2016
2,8853,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774,2019
3,8807,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282,2017


In [49]:
#Move the 'data_year'column from the last position to first
cols = list(crime_df.columns)
cols = [cols[-1]] + cols[:-1]
crime_df = crime_df[cols]
crime_df

,data_year,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,...,not_specified_recovered_count,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count
0,2018,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349
1,2016,8904,8904,0,8904,8904,8904,8904,8904,8904,...,0,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290
2,2019,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774
3,2017,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,0,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282


In [50]:
#Sort the year in ascending value
crime_df = crime_df.sort_values('data_year', ascending=True)
crime_df

,data_year,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,...,not_specified_recovered_count,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count
1,2016,8904,8904,0,8904,8904,8904,8904,8904,8904,...,0,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290
3,2017,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,0,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282
0,2018,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349
2,2019,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774


In [51]:
#Load crime data to SQL database
crime_df.to_sql(name='crime_data', con=engine, if_exists='replace', index=False)

In [52]:
#Query the database
pd.read_sql_query('select * from crime_data', con=engine)

,data_year,office_equipment,miscellaneous,not_specified,consumable_goods,locally_stolen_motor_vehicles,firearms,livestock,household_goods,jewelry_and_precious_metals,...,not_specified_recovered_count,consumable_goods_recovered_count,locally_stolen_motor_vehicles_recovered_count,firearms_recovered_count,livestock_recovered_count,household_goods_recovered_count,jewelry_and_precious_metals_recovered_count,currency_notes_etc_recovered_count,clothing_and_furs_recovered_count,telvisions_radio_stereos_etc_recovered_count
0,2016,8904,8904,0,8904,8904,8904,8904,8904,8904,...,0,3289894,706177670,916606,102456,1570109,5704667,3641961,6272342,2605290
1,2017,8807,8807,8807,8807,8807,8807,8807,8807,8807,...,0,2337220,744673819,1418204,88755,1550143,5634246,3146725,6210484,2487282
2,2018,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3004154,749684169,972729,111508,1354054,10771634,5402171,5626044,3827349
3,2019,8853,8853,8853,8853,8853,8853,8853,8853,8853,...,0,3052947,687681240,1509155,144162,1149992,5305881,6228195,6106406,2670774
